## IBM Db2 Event Store with Machine Learning & Model Deployment using Python API 

IBM Db2 Event Store is a hybrid transactional/analytical processing (HTAP) system. It extends the Spark SQL interface to support transactions and accelerate analytics queries. This notebook includes examples of using the Scala client interface to create a database and a table. It also shows how to insert and query data in IBM Db2 Event Store by using Spark SQL. This notebook shows how to build and deploy machine learning model using IBM Db2 Event Store.

When you finish this demo, you will know how to use Machine Learning and Model Deployment using IBM Db2 Event Store.

## Table of contents
1. [Connect to IBM Db2 Event Store](#connect-to-es)<br>
2. [Create a database and schema](#define-database)<br>
3. [Generate rows & insert data into Event Store table](#generate-insert-data)<br>
4. [Setup Spark Context](#setup-spark)<br>
5. [Open Database](#open-database)<br>
6. [Get table list from IBM Db2 Event Store ](#get-table-list)<br>
7. [Load IBM Db2 Event Store table](#load-table)<br>
8. [Run query on product table](#run-query)<br>
9. [Build Model](#build-model)<br>
10. [Save Model](#save-model)<br>
11. [Make a prediction](#make-prediction)<br>
   

In [ ]:
from eventstore.oltp import EventContext
from eventstore.oltp.row_generator import generate_tele
from eventstore.catalog import TableSchema, IndexSpecification, SortSpecification, ColumnOrder
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from eventstore.common import ConfigurationReader

<a id="connect-to-es"></a>
### 1. Set up connection to IBM Db2 Event Store

To establish a connection to IBM Db2 Event Store, you need to set the connection endpoints. Use the configuration reader to provide a set of APIs for IBM Db2 Event Store connection and configuration. The configurations need to be set differently, depending on whether the IBM Db2 Event Store is installed with Watson Studio Local (WSL) or IBM Cloud Private for Data (ICP4D).

For more details on setting up IBM Db2 Event Store connection in Jupyter Notebook, please read the official documentation:
https://www.ibm.com/support/knowledgecenter/en/SSGNPV_2.0.0/dsx/jupyter_prereq.html

#### For IBM Db2 Event Store installed with Watson Studio Local (WSL)
If your IBM Db2 Event Store is installed with Watson Studio Local (WSL), you will need to set the userID and password that will be used to connect to IBM Db2 Event Store instance.

By default, the connection will be estabilished to the IBM Db2 Event Store instance on the current Watson Studio Local cluster.

In [ ]:
# Using the configuration reader API, set up the userID and password that 
# will be used to connect to IBM Db2 Event Store.

ConfigurationReader.setEventUser("<userid>")
ConfigurationReader.setEventPassword("<password>")

#### For IBM Db2 Event Store installed with IBM Cloud Private for Data (ICP4D)
If your IBM Db2 Event Store is installed with IBM Cloud Private for Data (ICP4D), you will need to:
1. Add connection from IBM Cloud Private for Data (ICP4D) to IBM Db2 Event Store.  
For more details on adding connection, please read the official documentation:
https://www.ibm.com/support/knowledgecenter/en/SSGNPV_2.0.0/dsx/connect_ICP4D.html

2. Set the userID and password that will be used to connect to IBM Db2 Event Store instance.
3. Set the connection endpoint of the target IBM Db2 Event Store instance. The connection endpoint is in the format of "<JDBC_CONNECTION_ENDPOINT>;<SCALA_CONNECTION_ENDPOINT>". You can find the <JDBC_CONNECTION_ENDPOINT> and <SCALA_CONNECTION_ENDPOINT> in the **Database details** page in the IBM Cloud Private for Data UI console.  

For more details on setting up IBM Db2 Event Store connection in Jupyter Notebook, please read the official documentation: https://www.ibm.com/support/knowledgecenter/en/SSGNPV_2.0.0/dsx/jupyter_prereq.html

In [ ]:
## Note: Only run this cell if your IBM Db2 Event Store is installed with IBM Cloud Private for Data (ICP4D)

# Using the configuration reader API, set up the userID, password and connection endpoint that 
# will be used to connect to IBM Db2 Event Store.
ConfigurationReader.setConnectionEndpoints("<JDBC_CONNECTION_ENDPOINT>;<SCALA_CONNECTION_ENDPOINT>")
ConfigurationReader.setEventUser("<userid>")
ConfigurationReader.setEventPassword("<password>")

# Do not change the following configurations.
ConfigurationReader.setSslKeyStoreLocation("/user-home/_global_/security/customer-truststores/cacerts")
ConfigurationReader.setSslKeyStorePassword("changeit")
ConfigurationReader.setSslTrustStoreLocation("/user-home/_global_/security/customer-truststores/cacerts")
ConfigurationReader.setSslTrustStorePassword("changeit")
ConfigurationReader.setClientPluginName("IBMIAMauth")
ConfigurationReader.setClientPlugin(True)
ConfigurationReader.setSSLEnabled(True)

<a id="define-database"></a>
### 2. Connect to a database and schema 

In [ ]:
with EventContext.get_event_context("EVENTDB") as ctx:
    
    schema = StructType([
        StructField("PRODUCT_ID", IntegerType(), nullable=False),
        StructField("PRODUCT_CODE", IntegerType(), nullable=False),
        StructField("PRODUCT_DEPT", IntegerType(), nullable=True),
        StructField("PRODUCT_LINE", StringType(), nullable=True),
        StructField("GENDER", StringType(), nullable=True),
        StructField("AGE", IntegerType(), nullable=False),
        StructField("MARITAL_STATUS", StringType(), nullable=True),
        StructField("PROFESSION", StringType(), nullable=True)
    ])
    table_name = "product"
    table_schema = TableSchema(table_name, schema,
                              sharding_columns=["PRODUCT_ID"],
                              pk_columns=["PRODUCT_ID", "PRODUCT_CODE", "AGE"])
    index_spec = IndexSpecification(index_name="productidx",
                                    table_schema=table_schema,
                                    equal_columns=["PRODUCT_ID", "PRODUCT_CODE"],
                                    sort_columns=[SortSpecification("AGE", ColumnOrder.ASCENDING_NULLS_LAST)],
                                    include_columns=["PRODUCT_DEPT"])

    print("creating table with index...\n{}".format(table_schema))
    ctx.create_table_with_index(table_schema, index_spec)
    print("list of table names:")
    table_names = ctx.get_names_of_tables()
    for idx, name in enumerate(table_names):
        print("\t{}: {}".format(idx, name))

    print("get table: ")
    resolved_table_schema = ctx.get_table(table_name)
    print("resolved table schema: {}".format(resolved_table_schema))
    print("JVM resolved table schema: {}".format(resolved_table_schema.jresolved_table_schema))

<a id="generate-insert-data"></a>
### 3. Generate rows & insert data into Event Store table 

In [ ]:
row_batch = []
for x in range(0, 10000, 2):
    row_batch.append(dict(PRODUCT_ID=x, PRODUCT_CODE=1, PRODUCT_DEPT=50 , PRODUCT_LINE="Personal Accessories" , GENDER="Male", AGE=30, MARITAL_STATUS="Single", PROFESSION="Hospitality"))
    row_batch.append(dict(PRODUCT_ID=x, PRODUCT_CODE=2, PRODUCT_DEPT=60 , PRODUCT_LINE="Camping Equipment" , GENDER="Female", AGE=40, MARITAL_STATUS="Single", PROFESSION="Sales"))
    row_batch.append(dict(PRODUCT_ID=x, PRODUCT_CODE=3, PRODUCT_DEPT=70 , PRODUCT_LINE="Golf Equipment" , GENDER="Male", AGE=35, MARITAL_STATUS="Married", PROFESSION="Retail"))
    row_batch.append(dict(PRODUCT_ID=x, PRODUCT_CODE=4, PRODUCT_DEPT=80 , PRODUCT_LINE="Mountaineering Eq" , GENDER="Female", AGE=45, MARITAL_STATUS="Married", PROFESSION="Engineer"))
print(len(row_batch))
ctx.batch_insert(resolved_table_schema, row_batch)


In [ ]:
from pyspark.sql import SparkSession
from eventstore.sql import EventSession

<a id="setup-spark"></a>
### 4. Setup Spark Context

In [ ]:
sparkSession = SparkSession.builder.appName("Event Store ML in Python").getOrCreate()

eventSession = EventSession(sparkSession.sparkContext, "EVENTDB")
eventSession.set_query_read_option("SnapshotNow")

<a id="open-database"></a>
### 5. Open Database 

In [ ]:
eventSession.open_database()

<a id="get-table-list"></a>
### 6. Get table list from IBM Db2 Event Store 

In [ ]:
with EventContext.get_event_context("EVENTDB") as ctx:
    print("tables: ")
    table_names = ctx.get_names_of_tables()
    for idx, name in enumerate(table_names):
        print("\t{}: {}".format(idx, name))

<a id="load-table"></a>
### 7. Load IBM Db2 Event Store table 

In [ ]:
product = eventSession.load_event_table("product")
print("product table {}: {}".format(type(product), product))
product.createOrReplaceTempView("product")

<a id="run-query"></a>
### 8. Run query on product table 

In [ ]:
query = "SELECT PRODUCT_LINE, GENDER, AGE, MARITAL_STATUS, PROFESSION FROM product"
print("{}\nRunning query in Event Store...".format(query))
df_data = eventSession.sql(query)

In [ ]:
df_data.printSchema()
df_data.show(10)

<a id="build-model"></a>
### 9. Build Model 
In the model training process, the original dataset will be split into training dataset and testing dataset.

In [ ]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [ ]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

The following task is to set the input columns for model training, and use the corresponding algorithms to train the model.

In [ ]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [ ]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

In [ ]:
train_data.printSchema()

In [ ]:
print(train_data.count())

In [ ]:
model_rf = pipeline_rf.fit(train_data)

In [ ]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

<a id="save-model"></a>
### 10. Save Model

After the model is successfully trained, repository service is used to save the model. The model name and author information can be customized.

In [ ]:
from dsx_ml.ml import save
model_name = "Event Store Product Line Prediction Model"
saved_model = save(name=model_name, model=model_rf, test_data=train_data,algorithm_type='Classification')

In [ ]:
import os
import requests

header_online = {'Content-Type': 'application/json', 'Authorization': os.environ['DSX_TOKEN']}

print(saved_model['scoring_endpoint'])

In [ ]:
predictions = model_rf.transform(predict_data)
predictions.show(5)

In [ ]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

In [ ]:
import sys
import os
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go
init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

In [ ]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

In [ ]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()

product_data = [go.Pie(
            labels=cumulative_stats.index,
            values=cumulative_stats['GENDER'],
    )]

product_layout = go.Layout(
    title='Predicted product line client interest distribution',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

In [ ]:
age_data = [go.Bar(
            y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"],
            x=cumulative_stats.index
            
    )]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(
        title = "Product Line",
        showline=False,),
    yaxis=dict(
        title = "Mean AGE",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

<a id="make-prediction"></a>
### 11. Make a prediction

After deployment, the endpoint of model can be used to give prediction for new data using the online scoring service.

In [ ]:
new_data = { "GENDER" : "Female", "AGE" : 30, "MARITAL_STATUS" : "Married", "PROFESSION" : "Engineer" }
payload_scoring = [new_data]
scoring_response = requests.post(saved_model['scoring_endpoint'], json=payload_scoring, headers=header_online, verify=False)

print(scoring_response.text)

In [ ]:
new_data = { "GENDER" : "Male", "AGE" : 30, "MARITAL_STATUS" : "Married", "PROFESSION" : "Engineer" }
payload_scoring = [new_data]
scoring_response = requests.post(saved_model['scoring_endpoint'], json=payload_scoring, headers=header_online, verify=False)

print(scoring_response.text)

<a id="summary"></a>
## Summary
This demo introduced you to the IBM Db2 Event Store API for Machine Learning and Model Deployment. 

<hr>
Copyright &copy; IBM Corp. 2017. Released as licensed Sample Materials.